importing necessary liabraries

In [ ]:
import pandas as pd

> Now we are going to read the data using pandas funciton

In [ ]:
raw_data=pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')


In [ ]:
raw_data.describe()

In [ ]:
raw_data.shape

In [ ]:
raw_data.head()

availability is not important in this dataset.we will drop it

In [ ]:
data=raw_data.drop(['area_type','society'],axis='columns')
data.head()

Search for the Nan values and their counts

In [ ]:
data.isnull().sum()

We can simply delete those na values because our dataset is quite large

In [ ]:
data.dropna(inplace=True)
data.isnull().sum()

Now we can see that there are no nan values.so we can continue the operation

In [ ]:
data.head()

In the dataset in the size column we can see that there are many units like bedroom or bhk etc

In [ ]:
data['size'].unique()

as we can see that bhk and bedrooms are same so we will create a new column named as BHK and put the values from the size columns

In [ ]:
data['BHK']=data['size'].apply(lambda x:int(x.split(' ')[0]))
data.head()

so we don't need the size column. so we will drop this columns

In [ ]:
data.drop(['size','availability'],axis='columns',inplace=True)

In [ ]:
data.head()

In [ ]:
data['BHK'].unique()

In [ ]:
data.total_sqft.unique()

In our dataset total sqft contains some range values like 1133-1384. in this case we'll take their average.

We've made a function to convert the range values to float average value

In [ ]:
def is_total(x):
    try:
        
        token=x.split('-')
        if(len(token)==2):
            return (float(token[0])+float(token[1]))/2
        else:
            return float(x)
    except:
        return False
        
    


In [ ]:
data1=data.copy()

In [ ]:
data1['total_sqft']=data1['total_sqft'].apply(is_total)

In [ ]:
data1.head()

In [ ]:
data1.dtypes

In [ ]:
data1.loc[30]

# Price for square feet

In [ ]:
data1['total_sqft']=data1['total_sqft'].astype('float64')

In [ ]:
data1['price_per_sqft']=data1['price']*100000/data1['total_sqft']

In our dataset or any real estate bussiness location is a very important feature. we can't just remove it.First we need to check how many unique location we do have

In [ ]:
len(data1.location.unique())

now group by and count them on using aggregate function

In [ ]:
data1.location=data1.location.apply(lambda x:x.strip())
location_stat=data.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stat

what we gonna we gonna call location_stats<=10 as other place

In [ ]:
len(location_stat[location_stat<=10])

1028 location will be known as other location


In [ ]:
location_less10=location_stat[location_stat<=10]

In [ ]:
data1.location=data.location.apply(lambda x:'others' if x in location_less10 else x)
data1.tail()

In [ ]:
len(data1.location.unique())

In [ ]:
data1.head(500)

# Outlier removal

now we need some domain level knowledge.like for 12 rooms total sqft is 500 is not acceptable.so i ask my project manager what is the typical value of sqft per bedroom. so we will use a threshold that if per room bhk < 300 then. so the threshold for each room is 300 sqft

In [ ]:
data1[data1.total_sqft/data1.BHK < 300].head()

Look at the above data.3 rooms = 500 sqft is not possbile. so we need to remove them

In [ ]:
data2=data1[~(data1.total_sqft/data1.BHK < 300)]

In [ ]:
data2.total_sqft.describe()

Standard Deviation. The Standard Deviation is a measure of how spread out numbers are

we need to find out the mean and stadard deviation locationwise and remove price_persqft.filter out any data point beyond standard deviation

In [ ]:
import numpy as np

In [ ]:
def remove_out(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        print(key)
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduce=subdf[(subdf.price_per_sqft >(m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_out=pd.concat([df_out,reduce],ignore_index=True)
    return df_out


data3=remove_out(data2)
data3.shape



In [ ]:
data3.head()

In the dataset we have found that the price of 2bhk is greater than 3bhk of same price per area.This is an error so we need to remove them.Let's see it by using the plots

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as matplotlib


In [ ]:
def plot_s(df,location):
    bhk2=df[((df.location==location) & (df.BHK==2))]
    bhk3=df[((df.location==location) & (df.BHK==3))]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,color='red',label='3 BHK',s=50,marker='*')
    plt.xlabel('Total Square foot area')

    plt.ylabel('Price')
    plt.title('Location')
    plt.legend()
    
    
plot_s(data3,'Hebbal')
    
    

    

we can see that the price of 2 bhk flt is greater than 3bhk flat of same area.so we need to remove the outliers inorder to have a clean dataset

{

'1':{
    'mean':4000,
    'std':2000,
    'count':34
    },
'2':{
    'mean':4300,
    'std':2300,
    'count':22
    },
}

now we can remove those 2 bhk flats whose price_per_sqft is less than then the mean of price_per_sqft flat;


In [ ]:
def remove_bhk_outlier(df):
    exclude_indices=np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats={}
        for bhk,bhk_df in location_df.groupby('BHK'):
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df.price_per_sqft),
                'std':np.std(bhk_df.price_per_sqft),
                'count':bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('BHK'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

data4=remove_bhk_outlier(data3)
data4.shape[0]

In [ ]:
plot_s(data4,'Hebbal')


PLOT THE HISTOGRAM TO SEE THE DATA DISTRIBUTION

In [ ]:
matplotlib.rcParams['figure.figsize']= (20,10)
plt.hist(data4.price_per_sqft,rwidth=0.8)
plt.xlabel('Price per square feet')
plt.ylabel('count')

majority of 0-10k rupees 

In [ ]:
data4.bath.unique()

In [ ]:
data4[data4.bath>4]

for a single room there should be max 2 bathrooms

In [ ]:
matplotlib.rcParams['figure.figsize']= (20,10)
plt.hist(data4.bath,rwidth=0.8)
plt.xlabel('Bathroom')
plt.ylabel('count')

In [ ]:
data4[data4.bath<data4.BHK+2]

In [ ]:
data5=data4[data4.bath<data4.BHK+2]
data5.shape

In [ ]:
data5.columns

price_per_sqft,size can be droppped.These are not useful right on

In [ ]:
data6=data5.drop(['price_per_sqft','balcony'],axis='columns')
data6.head()

MODEL BUILDING


In [ ]:
data6.location.unique()

we need to use one hot encoding.with get dummies

In [ ]:
dum=pd.get_dummies(data6.location)

In [ ]:
data7=pd.concat([data6,dum.drop('others',axis='columns')],axis='columns')
data7.head()

In [ ]:
data7.drop('location',axis='columns',inplace=True)
data7.head()

As we've sucessfully cleaned our data.we need to split our data into input and target

In [ ]:
input=data7.drop('price',axis='columns')
target=data7.price

In [ ]:
target.head()

In [ ]:
input

we need to split the data into training and testing using train_test_split for linear regression model

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(input,target,test_size=0.2,random_state=10)

we need to import Linear regression from sklearn to  implement it

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

Now we'll try to solve it by using cross validation method

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),input,target,cv=cv)

There are various regression technique not only linear regression.we can use lasso,decision tree and various using Gidsearchcv method

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor


In [ ]:
input

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(input,target)

In [ ]:
X=input

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar',1000, 3, 2)

In [ ]:
 #export the model
import pickle
with open('./bangalore_house_price_prediction.pickle','wb') as f:
    pickle.dump(lr,f)

In [ ]:
import json
columns={
    'data_columns':[col.lower() for col in X.columns]
}
with open("./Columns.json","w") as f:
    f.write(json.dumps(columns))